# List of Packages to Download


In [ ]:
#!pip install transformers
#!pip install langchain-chroma
#!pip install langchain
#!pip install langchain_community
#!pip install langchainhub
#pip install langchain-huggingface
#pip install --upgrade --quiet  langchain sentence_transformers
#!pip install -U "huggingface_hub[cli]"
#!huggingface-cli login

# 1. Get a Data Loader


In [1]:
from langchain_community.document_loaders import WebBaseLoader


In [2]:
loader = WebBaseLoader("https://www.nba.com/player/2544/lebron-james")
data = loader.load()
data

[Document(page_content='LeBron James | Los Angeles Lakers | NBA.com\n\nNavigation ToggleNBANBA Draft HomeDraft BoardAll Picks: 1-58TeamsProspectsTrade TrackerDraft FAQFeaturedNBA TVHomeTickets2023-24 Season ScheduleU.S. Radio BroadcastersLeague Pass ScheduleLivestream ScheduleKey DatesTicketsHomeTop StoriesDraftFree AgencySummer LeagueOlympicsTransactionsFeaturesHistoryWriter ArchiveMoreStats HomePlayersTeamsLeadersStats 101Cume StatsLineups ToolMedia Central Game StatsDraftQuick LinksContact UsAtlanticBoston CelticsBrooklyn NetsNew York KnicksPhiladelphia 76ersToronto RaptorsCentralChicago BullsCleveland CavaliersDetroit PistonsIndiana PacersMilwaukee BucksSoutheastAtlanta HawksCharlotte HornetsMiami HeatOrlando MagicWashington WizardsNorthwestDenver NuggetsMinnesota TimberwolvesOklahoma City ThunderPortland Trail BlazersUtah JazzPacificGolden State WarriorsLA ClippersLos Angeles LakersPhoenix SunsSacramento KingsSouthwestDallas MavericksHouston RocketsMemphis GrizzliesNew Orleans Pel

# 2. Convert data to Vector Database


In [3]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings() #sentence-transformers/all-mpnet-base-v2


/opt/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
Chroma.from_documents(documents=data, embedding=embeddings_model, persist_directory="./chroma_db")

# 3. Make a RAG pipeline


In [5]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [6]:
#uncomment if using hugginface
#model_id = "google/gemma-2-9b"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id)
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
#hf = HuggingFacePipeline(pipeline=pipe)
#llm = hf

In [7]:


llm = ChatOllama(model="gemma2")

prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=HuggingFaceEmbeddings())

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [9]:
question = "Who's stats are these"
result = qa_chain({"query": question })

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [10]:
result["result"]

"This is a web page for LeBron James's profile on NBA.com. \n\nWould you like me to do anything with this information? For example, I could:\n\n* **Summarize LeBron James's career statistics.**\n* **Tell you about his recent games.**\n* **List his awards and honors.**\n* **Find information about a specific aspect of his career.**\n\n\n\nPlease let me know what you'd like me to do! \n"